In [1]:
import numpy as np
import gym
import matplotlib.pyplot as plt
from f110_gym.envs.base_classes import Integrator
from collections import Counter,defaultdict
import random
import os
import pickle
import pandas as pd
from collections import defaultdict,deque
import math
import cmath
import scipy.stats as stats
import time
from sklearn.preprocessing import StandardScaler
from scipy.stats import truncnorm
from sklearn.preprocessing import normalize

class Reward:
    def __init__(self, min_speed=0.5, max_speed=1.8, map_centers=None,track_width=2.2):
        self.min_speed = min_speed
        self.max_speed = max_speed
        self.map_centers = map_centers
        self.initial_point = np.array([[0, 0]])

        # Total track length
        self.track_lengths = [np.linalg.norm(self.map_centers[i, :] - self.map_centers[i + 1, :]) for i in range(self.map_centers.shape[0] - 1)]
        self.total_track_length = np.sum(self.track_lengths)

        # Hyperparameters
        self.track_width = 2.2
        self.epsilon = 1e-5
        self.distance_travelled = 0
        self.distance_multiplication_factor = 50
        self.distance_scaling_factor = 1.2
        self.min_distance_from_center = 0
        self.max_distance_from_center = 1
        self.centering_reward_parameter = 0.6

        # self.min_distance = self.exponential_distance(self.min_distance_from_center)
        # self.max_distance = self.exponential_distance(self.max_distance_from_center)

    def reset(self, point):
        self.distance_travelled = 0
        self.initial_point = point
    
    # def exponential_distance(self, distance):
    #     return self.distance_multiplication_factor * ((np.exp(-distance) / (2 - np.exp(-distance))) - self.centering_reward_parameter) + self.epsilon
    
    def exponential_angle(self, angle):
        func = lambda y : 2 * (np.exp(-0.017*y) - 0.5)
        return np.where(angle <= 90, func(angle),-1)

    def progress_reward(self, curr_position, next_position):
        distance = np.linalg.norm(curr_position - next_position)
        self.distance_travelled += distance
        return self.distance_travelled / self.total_track_length

    def calculate_distance_from_center(self, centers,curr):
        distances = np.linalg.norm(centers - curr, axis=1)
        return np.argmin(distances), distances[np.argmin(distances)]

    # def centering_reward(self, x, y):
    #     _, distance = self.calculate_distance_from_center(x, y)
    #     return self.distance_scaling_factor * self.exponential_distance(distance) / (self.min_distance - self.max_distance)

    def centering_reward(self, curr_state,next_state):
        # print(f'Current state: {curr_state}, Next state: {next_state}')
        curr_idx, c = self.calculate_distance_from_center(self.map_centers,curr_state)

        if curr_idx == len(self.map_centers) - 1:
            indices = [0, curr_idx-1]
        elif curr_idx == 0:
            indices = [curr_idx+1, len(self.map_centers)-1]
        else:
            indices = [curr_idx-1, curr_idx+1]

        subset_centers = self.map_centers[indices]

        next_idx, n = self.calculate_distance_from_center(subset_centers,next_state)

        curr_center = self.map_centers[curr_idx]
        next_center = subset_centers[next_idx]

        # print(f'Current closest center: {curr_center}, Next closest center: {next_center}')

        v1 = np.asarray([next_center[0] - curr_center[0], next_center[1] - curr_center[1]])
        v2 = np.asarray([next_state[0] - curr_state[0], next_state[1] - curr_state[1]])

        # print(f'Vectors are {v1} and {v2}')

        angle_rad = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
        angle = np.degrees(angle_rad)

        norm1 = np.linalg.norm(v1)
        norm2 = np.linalg.norm(v2)

        # print(f'Norms are {norm1} and {norm2}')

        # print(f'Angle rad is {angle_rad} and angle is {angle}')

        if norm1 ==0 or norm2 == 0:
            return -1

        return self.exponential_angle(angle)

    def calculate_reward(self, curr_state, next_state):
        progress_reward = self.progress_reward(curr_state, next_state)
        centering_reward = self.centering_reward(curr_state, next_state)
        # print(f"Distance reward: {progress_reward}, Centering reward: {centering_reward}")
        return progress_reward + centering_reward

In [2]:
class Reward:
    def __init__(self, min_speed=0.5, max_speed=1.8, map_centers=None,track_width = 2.2):
        self.min_speed = min_speed
        self.max_speed = max_speed
        self.map_centers = map_centers

        # Initial point and center that detrmines the position at the start of episode
        self.initial_point = np.array([[0, 0]])
        self.initial_center_idx , _ = self.__calculate_distance_from_center(self.map_centers,self.initial_point)
        self.initial_center = self.map_centers[self.initial_center_idx]

        # Race Track parameters
        self.distance_between_centers = np.hstack([[0.],np.linalg.norm(self.map_centers[:-1,:]- self.map_centers[1:,:],axis=1)])
        self.total_track_length = np.sum(self.distance_between_centers)
        self.track_width = track_width

        # Hyperparameters
        self.epsilon = 1e-5
        self.distance_travelled = 0

        # Centering reward function
        self.func = lambda y : 2 * (np.exp(-0.017*y) - 0.5)
    
    def __calculate_distance_from_center(self, centers,curr):
        distances = np.linalg.norm(centers - curr, axis=1)
        idx = np.argmin(distances)
        return idx, distances[idx]

    def reset(self, point):
        self.distance_travelled = 0
        self.initial_point = point
        idx , _ = self.__calculate_distance_from_center(self.map_centers,self.initial_point)
        self.initial_center_idx = idx
        self.initial_center = self.map_centers[self.initial_center_idx]

    def exponential_angle(self, angle):
        if angle <=90:
            return self.func(angle)
        return -1

    def progress_reward(self, curr_position, next_position):
        distance = np.linalg.norm(curr_position - next_position)
        self.distance_travelled += distance
        return self.distance_travelled / self.total_track_length


    def centering_reward(self, curr_position,next_position):
        
        position_vector = next_position - curr_position

        curr_idx, c = self.__calculate_distance_from_center(self.map_centers,curr_position)
        # print(f'Curr position: {curr_position}, Next position: {next_position} with initial center idx: {curr_idx}')

        if curr_idx == self.map_centers.shape[0] - 1:
            indices = [curr_idx-1,0]
        elif curr_idx == 0:
            indices = [self.map_centers.shape[0]-1,curr_idx+1]
        else:
            indices = [curr_idx-1, curr_idx+1]
            
        # print(f'Indices: {indices}')
        # print(np.vstack([self.map_centers[indices[0],:],self.map_centers[indices[1],:]]))

        next_idx, n = self.__calculate_distance_from_center(np.vstack([self.map_centers[indices[0],:],self.map_centers[indices[1],:]]),next_position)
        next_idx = indices[next_idx]
        # print(f'Next index is {next_idx} and distance is {n}')

        curr_center = self.map_centers[curr_idx]
        next_center = self.map_centers[next_idx]

        if curr_idx == next_idx:
            return -1
        
        # print(f'Current center: {curr_center}, Next center: {next_center}')
        center_vector = next_center - curr_center

        angle_rad = np.arccos(np.dot(center_vector, position_vector) / (np.linalg.norm(center_vector) * np.linalg.norm(position_vector)))

        # print(f'Center vector: {center_vector} and Position vector: {position_vector}')

        return self.exponential_angle(np.degrees(angle_rad))
    
    def milestone_reward(self, next_position):
        idx, _ = self.__calculate_distance_from_center(self.map_centers,next_position)
        if idx!=self.initial_center_idx:
            travelled = np.linalg.norm(self.initial_center - next_position) / self.total_track_length
            if  travelled >= np.abs(self.distance_between_centers[idx] - self.distance_between_centers[self.initial_center_idx]) / self.total_track_length :
                self.initial_center_idx = idx
                self.initial_center = self.map_centers[idx]
                return 5
        return 0

    def calculate_reward(self, curr_position, next_position):
        progress_reward = self.progress_reward(curr_position, next_position)
        centering_reward = self.centering_reward(curr_position, next_position)
        milestone_reward = self.milestone_reward(next_position)
        # print(f"Distance reward: {progress_reward}, Centering reward: {centering_reward}, Milestone reward: {milestone_reward}")
        return progress_reward + centering_reward + milestone_reward

In [3]:
class IndexSelector:
    def __init__(self, num_indices):
        self.num_indices = num_indices
        self.visited_indices = set()
        self.probabilities = np.ones(num_indices) / num_indices

    def select_index(self):
        if len(self.visited_indices) == self.num_indices:
            # Reset the probabilities and visited indices
            print('Visited all indices, resetting')
            self.visited_indices = set()
            self.probabilities = np.ones(self.num_indices) / self.num_indices

        # Select an index based on the current probabilities
        random_idx = np.random.choice(np.arange(self.num_indices), p=self.probabilities)

        # Update the probabilities
        self.visited_indices.add(random_idx)
        if len(self.visited_indices) < self.num_indices:
            self.probabilities[random_idx] = 0
            remaining_prob = 1 - np.sum(self.probabilities)
            self.probabilities[self.probabilities > 0] += remaining_prob / np.sum(self.probabilities > 0)

        return random_idx

In [4]:
class F1Tenth_navigation:

    def __init__(self,gym_env_code='f110_gym:f110-v0', num_agents=1, map_path='./f1tenth_racetracks/Austin/Austin_map', map_ext='.png', sx=0., sy=0., stheta=0., map_centers_file=None, save_path=None, track_name=None, inference=None,reward_file=None,collision_file=None):

        # Environment setup
        self.env = gym.make(gym_env_code, map=map_path, map_ext=map_ext, num_agents=num_agents, timestep=0.01, integrator=Integrator.RK4)
        self.env.add_render_callback(self.render_callback)
        self.sx, self.sy, self.stheta = sx, sy, stheta
        self.save_path = save_path
        self.track_name = track_name
        self.num_agents = num_agents
        self.map_path = map_path
        self.map_ext = map_ext
        self.map_centers_file = pd.read_csv(map_centers_file)
        self.map_centers_file.columns = ['x', 'y', 'w_r', 'w_l']
        self.map_centers_file.index = self.map_centers_file.index.astype(int)
        self.map_centers = self.map_centers_file.values[:, :2]
        self.track_width = self.map_centers_file.loc[0,'w_r'] + self.map_centers_file.loc[0,'w_l']
        self.reward_file = reward_file
        self.collision_file = collision_file

        # Random Seed
        self.random_seed = 42
        np.random.seed(self.random_seed)

        # Environment Observation Parameters
        self.num_beams = 1080
        self.n_features = 11
        self.angle = 270

        # Action Space Parameters
        self.num_angles = 31
        self.num_speeds = 10

        # LiDAR downsampling parameters
        self.n_sectors = 30

        # State Space Parameters
        self.num_states = 2 ** self.n_features

        # Speed Parameters
        self.min_speed = 0.8
        self.max_speed = 2

        # Action Space
        self.angles_deg = np.linspace(-self.angle // 2, self.angle // 2, self.num_angles)[::-1]
        self.angles = self.convert_deg_to_rad(self.angles_deg)
        self.speeds = np.linspace(self.min_speed, self.max_speed, self.num_speeds)
    
        # State Space - Q-Table
        if inference is not None:
            self.weights = np.load(inference)
            self.num_collisions = int(inference.split('_')[1].split('.')[0])
            print(f'Loaded Weights')
        else:
            self.weights = np.zeros((self.num_states,self.num_angles,self.num_speeds))
            self.num_collisions = 0


        # ELigibility Trace
        self.ET_IS = np.zeros((self.num_states,self.num_angles,self.num_speeds))

        # projection matrix
        self.projection_matrix = self.generate_projection_matrix()

        # binary powers
        self.binary_powers = np.array([2 ** i for i in range(self.n_features)])

        # Training Variables
        self.curr_state = None
        self.next_state = None
    
        
        self.action_threshold_decay = 0.99997
        self.action_threshold = 0.20 * (self.action_threshold_decay ** self.num_collisions)

        # Imported Classes
        self.reward_class = Reward(min_speed=self.min_speed, max_speed=self.max_speed, map_centers=self.map_centers, track_width=self.track_width)
        self.index_selector = IndexSelector(self.map_centers.shape[0])      

        # Sarsa Parameters
        self.learning_rate = 0.1
        self.discount_factor = 0.95
        self.decay_rate = 0.9

        # Reward
        self.reward = 0
        self.episode_reward = 0
        self.cumulative_reward = 0
        self.episodic_rewards = [0]
        self.delayed_reward_counter = 0

        # Time
        self.collision_times = [0]
        

    def render_callback(self, env_renderer):
        e = env_renderer
        x = e.cars[0].vertices[::2]
        y = e.cars[0].vertices[1::2]
        top, bottom, left, right = max(y), min(y), min(x), max(x)
        e.score_label.x = left
        e.score_label.y = top - 700
        e.left = left - 800
        e.right = right + 800
        e.top = top + 800
        e.bottom = bottom - 800

    def convert_deg_to_rad(self, array):
        return array * np.pi / 180
    
    # def get_statistical_properties(self,lidar_input,n_sectors=30):
    #     lidar_input = np.asarray(lidar_input, dtype=np.float32)
    #     sector_size = lidar_input.shape[0] // n_sectors
    #     sectors = lidar_input[:sector_size * n_sectors].reshape(n_sectors, sector_size)
    #     sector_features = np.vstack(
    #         [   np.mean(sectors, axis=1),
    #             np.std(sectors, axis=1),
    #             np.max(sectors, axis=1),
    #             np.min(sectors, axis=1),
    #             np.median(sectors, axis=1),
    #             np.percentile(sectors, 75, axis=1)
    #         ])
    #     return sector_features
    def get_statistical_properties(self,lidar_input,n_sectors=30):
        lidar_input = np.asarray(lidar_input, dtype=np.float32)
        sector_size = lidar_input.shape[0] // n_sectors
        sectors = lidar_input[:sector_size * n_sectors].reshape(n_sectors, sector_size)
        medians = np.median(sectors, axis=1)

        return medians.reshape(1, -1)
    
    # def binarize_vector(self,vector):
    #     minimum,maximum = vector.min(),vector.max()
    #     threshold = minimum + (maximum - minimum)/2
    #     binary_vector = np.where(vector >= threshold,1,0)
    #     return binary_vector
    def binarize_vector(self,vector):
        mi,ma = np.min(vector),np.max(vector)
        threshold = (ma+mi)/2
        return np.where(vector > threshold, 1, 0)
    
    # def generate_projection_matrix(self):
    #     a = (0 - 0.5) / 0.25  
    #     b = (1 - 0.5) / 0.25   
    #     return truncnorm.rvs(a, b, loc=0.5, scale=0.25, size=(self.n_sectors, self.n_features))

    def generate_projection_matrix(self):
        # Generate a random matrix with values 0 and 1 based on the given probabilities [prob_0,prob_1]
        matrix = np.random.choice([0, 1], size=(self.n_sectors, self.n_features), p=[0.5, 0.5])
        return matrix
    
    def get_binary_representation(self,lidar_input):
        lidar = normalize(self.get_statistical_properties(lidar_input),axis=1)
        proj = np.dot(lidar,self.projection_matrix)
        binary = self.binarize_vector(proj)
        return binary
    
    def get_state(self, binary):
        return np.dot(binary[0], self.binary_powers)
    
    def select_action(self, state):
        random_number = np.random.rand()
        if random_number < self.action_threshold:
            angle_index = np.random.randint(0, self.num_angles)
            speed_index = np.random.randint(0, self.num_speeds)
        else:
            flattened = self.weights[state].flatten()
            max_value = np.max(flattened)
            max_indices = np.where(flattened == max_value)[0]
            max_index = np.random.choice(max_indices)
            
            angle_index, speed_index = np.unravel_index(max_index, self.weights[state].shape)

        self.action_threshold *= self.action_threshold_decay

        return angle_index, speed_index

    def select_action_inference(self, state):
        
        flattened = self.weights[state].flatten()
        max_value = np.max(flattened)
        max_indices = np.where(flattened == max_value)[0]
        max_index = np.random.choice(max_indices)
        
        angle_index, speed_index = np.unravel_index(max_index, self.weights[state].shape)

        return angle_index, speed_index
    
    def sarsa_weight_update(self,angle_idx,speed_idx,reward):
        next_angle_idx,next_speed_idx = self.select_action(self.next_state)
        delta = reward + self.discount_factor * self.weights[self.next_state,next_angle_idx,next_speed_idx] - self.weights[self.curr_state,angle_idx,speed_idx]

        self.weights += self.learning_rate * delta * self.ET_IS
        return next_angle_idx,next_speed_idx
    
    def set_eligibility_trace(self,angle_idx,speed_idx):
        self.ET_IS [self.curr_state,angle_idx,speed_idx] = 1

    def decay_eligibility_trace(self):
        self.ET_IS *= self.discount_factor * self.decay_rate

    def save_reward_time(self):
        if not os.path.exists(os.path.join(self.save_path, self.track_name)):
            os.mkdir(os.path.join(self.save_path, self.track_name))
        
        if self.reward_file is not None:

            r = np.append(np.load(self.reward_file), self.episodic_rewards[-1])
            t = np.append(np.load(self.collision_file), self.collision_times[-1])
            np.save(os.path.join(self.save_path, self.track_name, f'rewards.npy'), np.array(r))
            np.save(os.path.join(self.save_path, self.track_name, f'times.npy'), np.array(t))
        else:
            np.save(os.path.join(self.save_path, self.track_name, f'rewards.npy'), np.array(self.episodic_rewards))
            np.save(os.path.join(self.save_path, self.track_name, f'times.npy'), np.array(self.collision_times))
    
    def save_weights(self):
        if not os.path.exists(os.path.join(self.save_path, self.track_name)):
            os.mkdir(os.path.join(self.save_path, self.track_name))
        np.save(os.path.join(self.save_path, self.track_name, f'weights_{self.num_collisions + 1}.npy'), self.weights)
        # print(f'File saved')


    def train(self):
        obs, step_reward, done, info = self.env.reset(np.array([[self.sx, self.sy, self.stheta]]))
        lidar = obs['scans'][0]

        down_sampled = self.get_binary_representation(lidar)
        self.curr_state = self.get_state(down_sampled)
        self.reward_class.reset(np.array([[self.sx, self.sy]]))
        angle_index,speed_index = self.select_action(self.curr_state)
        start_time = time.time()
        while True:
            # np.save(f'./LiDAR_scans/scan_{self.curr_state}.npy',lidar)
            steering_angle,speed = self.angles[angle_index],self.speeds[speed_index]
            curr_x = obs['poses_x'][0]
            curr_y = obs['poses_y'][0]
            obs, reward, done, info = self.env.step(np.array([[steering_angle, speed]]))
            lidar = obs['scans'][0]
            down_sampled = self.get_binary_representation(lidar)
            self.next_state = self.get_state(down_sampled)

            if done:
                self.reward = -50
            else:           
                self.reward = self.reward_class.calculate_reward(np.array([curr_x, curr_y]), np.array([obs['poses_x'][0], obs['poses_y'][0]]))
            # print(f'Reward: {self.reward},Curr_state is {self.curr_state},steering angle: {steering_angle}, speed: {speed}')
            # print(f'State: {self.curr_state}, Next state: {self.next_state}, Reward: {self.reward}')
            self.cumulative_reward += self.reward
            self.episode_reward += self.reward
            self.set_eligibility_trace(angle_index,speed_index)
            if self.delayed_reward_counter == 10:
                angle_index,speed_index = self.sarsa_weight_update(angle_index,speed_index,self.episode_reward)
                self.delayed_reward_counter = 0
                # print(f'Episode reward is {self.episode_reward}')
            else:
                self.delayed_reward_counter += 1
                angle_index,speed_index = self.sarsa_weight_update(angle_index,speed_index,0)
               
            
            self.decay_eligibility_trace()
            self.curr_state = self.next_state

            # Randomize the starting point after collision
            if done:
                _,_ = self.sarsa_weight_update(angle_index,speed_index,self.episode_reward)
                self.num_collisions += 1
                self.episode_reward = 0

                # Obtaining a new random position on the track
                random_idx = self.index_selector.select_index()
                n_x, n_y = self.map_centers[random_idx]
                delta_x, delta_y = np.random.uniform(-0.75, 0.75), np.random.uniform(-0.2, 0.2)
                n_theta = np.random.choice(self.angles)

                # Sensing the new state
                obs, step_reward, done, info = self.env.reset(np.array([[n_x + delta_x, n_y + delta_y, n_theta]]))
                lidar = obs['scans'][0]
                down_sampled = self.get_binary_representation(lidar)
                self.curr_state = self.get_state(down_sampled)
                angle_index,speed_index = self.select_action(self.curr_state)

                # Resetting 
                self.reward_class.reset(np.array([[n_x + delta_x, n_y + delta_y]]))
                self.ET_IS.fill(0)

                # Checkpoint
                if (self.num_collisions + 1) % 1000 == 0:
                    end_time = time.time()
                    print(f'Collision: {self.num_collisions+1}, Time: {end_time - start_time}, Reward: {self.cumulative_reward - self.episodic_rewards[-1]}')
                    self.collision_times.append(end_time - start_time)
                    self.episodic_rewards.append(self.cumulative_reward - self.episodic_rewards[-1])
                    self.save_reward_time()
                    start_time = end_time
                    self.save_weights()

                # if (self.num_collisions+1) % 5000 == 0:
                #     self.save_weights()

            # self.env.render(mode='human')
    
    def inference(self):
        obs, step_reward, done, info = self.env.reset(np.array([[self.sx, self.sy, self.stheta]]))
        lidar = obs['scans'][0]
        down_sampled = self.get_binary_representation(lidar)
        self.curr_state = self.get_state(down_sampled)
        self.reward_class.reset(np.array([[self.sx, self.sy]]))
        angle_index,speed_index = self.select_action_inference(self.curr_state)
        while not done:
            steering_angle,speed = self.angles[angle_index],self.speeds[speed_index]
            obs, reward, done, info = self.env.step(np.array([[steering_angle, speed]]))
            lidar = obs['scans'][0]
            down_sampled = self.get_binary_representation(lidar)
            self.next_state = self.get_state(down_sampled)
            angle_index,speed_index = self.select_action_inference(self.next_state)
            self.curr_state = self.next_state
            
            self.env.render(mode='human')

In [ ]:
path = './f1tenth_racetracks'
all_map_paths=[]
map_centers = []
map_names = []
track_lengths=[]
for folder in os.listdir(path):
    if folder not in ['README.md','.gitignore','convert.py','LICENSE','rename.py','.git']:
        folder_name=folder
        file_name=folder_name.replace(' ','')+'_map'
        map_center = folder_name.replace(' ','')+'_centerline.csv'
        track_lengths.append(len(pd.read_csv(f'{path}/{folder_name}/{map_center}')))
        map_names.append(folder_name)
        all_map_paths.append(f'{path}/{folder_name}/{file_name}')
        map_centers.append(f'{path}/{folder_name}/{map_center}')

list(zip(map_names,track_lengths))

In [ ]:
global num_agents,map_path,map_ext,sx,sy,stheta
num_agents = 1
map_ext = '.png'
sx = 0.
sy = 0.
stheta = 1.0
map_index = 1
map_path = all_map_paths[map_index]
map_center = map_centers[map_index]
map_name = map_names[11]
save_path = './Weights/'
# inference_file = None
# reward_file = None
# collision_file = None
inference_file = f'./Weights/{map_name}/weights_38000.npy'
reward_file = f'./Weights/{map_name}/rewards.npy'
collision_file = f'./Weights/{map_name}/times.npy'
simulator = F1Tenth_navigation(num_agents=num_agents,map_path=map_path,map_ext=map_ext,sx=sx,sy=sy,stheta=stheta,map_centers_file=map_center,save_path=save_path,track_name=map_name,inference=inference_file,reward_file=reward_file,collision_file=collision_file)
# simulator.train()
simulator.inference()

# Nuerburgring
Best runs 31k-> reward:+37k reward; 39k -> reward:+94k , 40k ->rewrad:215k ,15k , 27k

In [ ]:
reward = np.load('/workspaces/f1_tenth/Weights/Nuerburgring/rewards.npy')
times = np.load('/workspaces/f1_tenth/Weights/Nuerburgring/times.npy')

In [8]:
path = '/workspaces/f1_tenth/Weights/Nuerburgring'
episode_num=[0]
for file in os.listdir(path):
    if file.startswith('weights'):
        episode_num.append(int(file.split('_')[1].split('.')[0]))

episode_num = sorted(episode_num) 

In [ ]:
reward

In [ ]:
plt.plot(episode_num,reward)
plt.title('Reward Vs Episode')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()

In [ ]:
times

In [ ]:
plt.plot(episode_num,times)
plt.title('Time Vs Episode')
plt.xlabel('Episode')
plt.ylabel('Time to collision')
plt.show()